In [36]:
import os
import sys
import dspy

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)


In [51]:
import duckdb
from functools import wraps
from datetime import datetime

DB_PATH = "logs.db"

def initialize_db():
    conn = duckdb.connect(DB_PATH)
    conn.execute("""
    CREATE TABLE IF NOT EXISTS logs (
        id BIGINT,
        timestamp TIMESTAMP,
        user_query TEXT,
        generated_response TEXT
    )
    """)
    conn.close()

initialize_db()

In [52]:
def log_conversation_to_db(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        
        user_query = kwargs.get('user_query', args[0] if args else None)
        generated_response = result
        
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        conn = duckdb.connect(DB_PATH)
        conn.execute("""
            INSERT INTO logs (timestamp, user_query, generated_response)
            VALUES (?, ?, ?)
        """, (timestamp, user_query, generated_response))
        
        print(" =^=^= Conversation logged successfully. =^=^=")        
        conn.close()
        return result
    return wrapper

In [53]:
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.environ["GROQ_API_KEY"]
openai_api_key = os.environ["OPENAI_API_KEY"]

In [54]:
mix = dspy.GROQ(model="mixtral-8x7b-32768", api_key=groq_api_key)
openai = dspy.OpenAI(model="gpt-4o-mini", api_key=openai_api_key)
dspy.settings.configure(lm=mix)

In [11]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factual answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

pot = dspy.ProgramOfThought(GenerateAnswer)

question = 'Sarah has 5 apples. She buys 7 more apples from the store. How many apples does Sarah have now?'
result = pot(question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after ProgramOfThought process): {result.answer}")

Question: Sarah has 5 apples. She buys 7 more apples from the store. How many apples does Sarah have now?
Final Predicted Answer (after ProgramOfThought process): 12


In [48]:
import IPython.display as display_module

class CareerGuider(dspy.Signature):
    """
    Provide career guidance based on the user's input.
    Offer advice on career transitions, required skills, and industry trends.
    Make sure the guidance is personalized and actionable.
    """

    user_query = dspy.InputField(desc="The user's question about their career path or job opportunities.")
    result = dspy.OutputField(desc="Career guidance based on the user's query.")

guide_career = dspy.ChainOfThought(CareerGuider)

query = """Provide career guidance for someone interested in transitioning into AI from a software engineering background.
Include recommended skills, industry trends, and potential career paths.
Guidance should be insightful, coherent, and practical for someone with 3-5 years of experience."""

career_guidance = guide_career(user_query=query)

display_module.display_markdown(career_guidance.result, raw=True)

1. Skills to Develop:
   - Deep Learning Frameworks: Familiarize yourself with deep learning frameworks such as TensorFlow, PyTorch, and Keras. These tools are essential for building and training AI models.
   - Machine Learning Algorithms: Strengthen your understanding of machine learning algorithms, including supervised, unsupervised, and reinforcement learning techniques.
   - Data Manipulation and Analysis: Improve your skills in data preprocessing, cleaning, and analysis using libraries like Pandas, NumPy, and

In [55]:
@log_conversation_to_db
def get_result(user_query):
    return guide_career(user_query=query).result

In [56]:
response = get_result(user_query=query)
print(response)

 =^=^= Conversation logged successfully. =^=^=
1. Skills to Develop:
   - Deep Learning Frameworks: Familiarize yourself with deep learning frameworks such as TensorFlow, PyTorch, and Keras. These tools are essential for building and training AI models.
   - Machine Learning Algorithms: Strengthen your understanding of machine learning algorithms, including supervised, unsupervised, and reinforcement learning techniques.
   - Data Manipulation and Analysis: Improve your skills in data preprocessing, cleaning, and analysis using libraries like Pandas, NumPy, and
